# Run all chromosomes without Bottleneck - maximum recon?

- Read in best model from fine-grained
- Rerun

In [1]:
import os
import pandas as pd
import numpy as np
import json
import pickle 
import torch

# Find best models

In [7]:
PATH_data = "/data/scratch/skatz/PROJECTS/methylnet/1_healthyVAE/data/GSE87571/train_val_test_sets/"

In [14]:
for CHR in [f"chr{i}" for i in range(1,23)]:
    PATH_results = f"logs/finalModels/{CHR}/noBottleneck"
    os.makedirs(PATH_results, exist_ok=True)
    param_grid = dict()

    ### Step 1: read in old model
    ## a. original parameters
    with open(f"logs/optimisation/{CHR}/fine/param_grid_fine.json", "r") as f: dict_oldGrid = json.load(f)
    ## b. best HP model
    with open(f"logs/optimisation/{CHR}/fine/best_model_fineOptimization.json", "r") as f: dict_bestModel = json.load(f)

    ### Step 3: design latSizes
    with open(os.path.join(PATH_data, f"{CHR}_train_methyl_array.pkl"), "rb") as f: train_dataset = pickle.load(f) #
    num_cpgs = train_dataset["beta"].shape[1]
    param_grid["latentSize"] = num_cpgs
    ### Step 4: get lr and dropout
    param_grid["lr"] = dict_bestModel["lr"]
    param_grid["dropout"] = dict_bestModel["dropr"]

    ### Save parameter grid in file for later documentation
    with open(f"{PATH_results}/param_grid.json", "w") as f: f.write(json.dumps(param_grid, indent="\t"))

In [15]:
for CHR in [f"chr{i}" for i in range(1,23)]:
    PATH_results = f"logs/finalModels/{CHR}/noBottleneck"
    ### Load parameter grid
    with open(f"{PATH_results}/param_grid.json", "r") as f: param_grid_comb=json.loads(f.read())

    ### Generate submit.sh with combinations
    os.makedirs(f"{PATH_results}/submit", exist_ok=True)
    with open("submit_template_noEncoder.sh", "r") as f: template=f.read()             
    latSize = str(param_grid_comb["latentSize"])
    lr = str(param_grid_comb["lr"])
    dropr = str(param_grid_comb["dropout"])

    ### Generate run name - combination of parameter settings
    fileName = f"latSize_{latSize}"
    ### Replace in template file
    template_updated = template.replace("$PATH", str(PATH_results+"/"+fileName)) \
                                   .replace("$CHR",  str(CHR)) \
                                   .replace("$LATSIZE", latSize) \
                                   .replace("$LR", lr) \
                                   .replace("$DROPR", dropr)
    with open(f"{PATH_results}/submit/{fileName}.sh", "w") as f: f.write(template_updated)
    print(f"Wrote file \t{fileName}")

Wrote file 	latSize_29482
Wrote file 	latSize_21984
Wrote file 	latSize_15547
Wrote file 	latSize_13100
Wrote file 	latSize_15965
Wrote file 	latSize_23070
Wrote file 	latSize_19599
Wrote file 	latSize_13704
Wrote file 	latSize_6619
Wrote file 	latSize_15736
Wrote file 	latSize_18570
Wrote file 	latSize_15165
Wrote file 	latSize_7802
Wrote file 	latSize_9417
Wrote file 	latSize_9715
Wrote file 	latSize_13823
Wrote file 	latSize_17044
Wrote file 	latSize_3654
Wrote file 	latSize_15475
Wrote file 	latSize_6567
Wrote file 	latSize_2719
Wrote file 	latSize_5243


# Analysis

In [15]:
from scipy.stats import pearsonr
def calc_r2(model, data_tensor):    
    model.eval()
    orig = data_tensor.cpu().detach().numpy()
    recon = model(data_tensor)
    # check if VAE or AE was used
    if isinstance(recon, tuple):
        recon = recon[0].cpu().detach().numpy()
    else:
        recon = recon.detach().numpy()
    r2 = []
    for i in range(recon.shape[1]):
        r2.append(pearsonr(orig[:,i], recon[:,i])[0])
    return np.array(r2).mean().round(3), np.array(r2).std().round(3)

In [27]:
PATH_data = "/data/scratch/skatz/PROJECTS/methylnet/1_healthyVAE/data/GSE87571/train_val_test_sets/"

all_mean = []
all_std = []
for CHR in [f"chr{i}" for i in range(1,23)]:
    with open(os.path.join(PATH_data, f"{CHR}_test_methyl_array.pkl"), "rb") as f: test_dataset = pickle.load(f) #
    test_tensor = torch.tensor(test_dataset["beta"].values, dtype=torch.float32)
    
    PATH_results = f"logs/finalModels/{CHR}/noBottleneck"
    with open(f"{PATH_results}/param_grid.json", "r") as f: dict_bestModel = json.load(f)
    latSize = dict_bestModel["latentSize"]
    name = f"latSize_{latSize}"
    path = f"{PATH_results}/{name}"
    model = torch.load(f"{path}/checkpoint/trainedModel.pth", map_location=torch.device('cpu'))
    
    mean, std = calc_r2(model, test_tensor)
    print(CHR, mean, std)
    all_mean.append(mean)
    all_std.append(std)

chr1 0.64 0.183
chr2 0.648 0.179
chr3 0.671 0.17
chr4 0.642 0.181
chr5 0.638 0.181
chr6 nan nan
chr7 0.641 0.179
chr8 0.646 0.177
chr9 nan nan
chr10 0.643 0.179
chr11 0.656 0.173
chr12 0.646 0.178
chr13 nan nan
chr14 0.653 0.177
chr15 0.646 0.179
chr16 0.643 0.174
chr17 0.667 0.171
chr18 0.677 0.176
chr19 0.646 0.177
chr20 nan nan
chr21 0.687 0.169
chr22 0.662 0.173


In [33]:
print(np.nanmean(np.array(all_mean)))
print(np.nanmean(np.array(all_std)))

0.652888888888889
0.17644444444444443
